In [14]:
import os
import pandas as pd
import numpy as np
import xgboost
import matplotlib.pyplot as plt
from xgboost import plot_importance
from sklearn import metrics

In [15]:
import pandas as pd
# Load the dataset
dataframe = pd.read_csv('Petrol CSV w Date.csv')
# Convert 'Date' column to DatetimeIndex with the correct format
dataframe['date'] = pd.to_datetime(dataframe['date'], format='%Y')
# Now you can work with the DataFrame as needed
print(dataframe.head())

        date  PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit
0 1980-01-01                                           542254.0         
1 1981-01-01                                           589017.0         
2 1982-01-01                                           644907.0         
3 1983-01-01                                           690214.0         
4 1984-01-01                                           754725.0         


In [16]:
df1 = dataframe.copy()
del df1['date']
df1

,PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit
0,542254.000
1,589017.000
2,644907.000
3,690214.000
4,754725.000
5,783807.000
6,822735.000
7,874255.000
8,940358.000
9,989211.000


In [17]:
print(dataframe)

         date  PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit
0  1980-01-01                                         542254.000         
1  1981-01-01                                         589017.000         
2  1982-01-01                                         644907.000         
3  1983-01-01                                         690214.000         
4  1984-01-01                                         754725.000         
5  1985-01-01                                         783807.000         
6  1986-01-01                                         822735.000         
7  1987-01-01                                         874255.000         
8  1988-01-01                                         940358.000         
9  1989-01-01                                         989211.000         
10 1990-01-01                                        1069662.000         
11 1991-01-01                                        1066161.000         
12 1992-01-01                         

In [18]:
df = df1[:44]
df

,PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit
0,542254.000
1,589017.000
2,644907.000
3,690214.000
4,754725.000
5,783807.000
6,822735.000
7,874255.000
8,940358.000
9,989211.000


In [19]:
split_date = 34
train = df.loc[df.index <= split_date].copy()
test = df.loc[df.index > split_date].copy()

In [20]:
len(train)

35

In [21]:
len(test)

9

In [22]:
df.shape

(44, 1)

In [23]:
def create_features(df, label=None):
    """
    Creates time series features from datetime index
    """
    # Ensure the index is datetime
    if not pd.api.types.is_datetime64_any_dtype(df.index):
        df.index = pd.to_datetime(df.index)
    
    # Create a 'date' column from the index
    df['date'] = df.index
    
    # Ensure 'date' column is datetime
    if not pd.api.types.is_datetime64_any_dtype(df['date']):
        df['date'] = pd.to_datetime(df['date'])

    # Extract datetime features
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.isocalendar().week  # Use isocalendar for week numbers

    X = df[['hour', 'dayofweek', 'quarter', 'month', 'year',
            'dayofyear', 'dayofmonth', 'weekofyear']]
    if label:
        y = df[label]
        return X, y
    return X


# Create features and labels for training data
X_train, y_train = create_features(train, label='PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit')

# Create features and labels for test data
X_test, y_test = create_features(test, label='PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit')

## moavia Code

In [24]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
# Define a parameter grid for hyperparameter tuning
param_grid = {
 'max_depth': [3, 4, 5, 6],
 'learning_rate': [0.01, 0.02, 0.03],
 'n_estimators': [150, 200, 250],
 'colsample_bytree': [0.5, 0.7, 0.9]
}
# Create an XGBoost regressor with 'reg:squarederror' as the objective
xgb = XGBRegressor(objective='reg:squarederror')
# Create a GridSearchCV object with custom scoring functions
scoring = {
 'MSE': 'neg_mean_squared_error',
 'MAE': 'neg_mean_absolute_error',
 'MAPE': 'neg_mean_absolute_percentage_error',
 'RMSE': 'neg_root_mean_squared_error'
}
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring=scoring, cv=4, refit='MSE', verbose=1)
# Fit the grid search to your data
grid_search.fit(X_train, y_train)
# Get the best hyperparameters from the grid search
best_params = grid_search.best_params_
# Create a new XGBoost model with the best hyperparameters
best_xgb = XGBRegressor(objective='reg:squarederror', 
                        **best_params,
                        early_stopping_rounds=50,)





# Fit the model with early stopping
best_xgb.fit(X_train, y_train,
 eval_set=[(X_train, y_train), (X_test, y_test)],
 verbose=None)
# Evaluate the model on the test data
y_pred = best_xgb.predict(X_test)
# Calculate the evaluation metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100
mpe = ((y_test - y_pred) / y_test).mean() * 100
rmse = mean_squared_error(y_test, y_pred, squared=False)
print("Best Hyperparameters:", best_params)
print("Mean Squared Error (MSE):", mse)
print("Mean Absolute Error (MAE):", mae)
print("Mean Absolute Percentage Error (MAPE):", mape)
print("Mean Percentage Error (MPE):", mpe)
print("Root Mean Squared Error (RMSE):", rmse)

Fitting 4 folds for each of 108 candidates, totalling 432 fits


Best Hyperparameters: {'colsample_bytree': 0.5, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150}
Mean Squared Error (MSE): 36771017388237.04
Mean Absolute Error (MAE): 5931803.984222222
Mean Absolute Percentage Error (MAPE): 81.17750591573298
Mean Percentage Error (MPE): 81.17750591573298
Root Mean Squared Error (RMSE): 6063911.063681346


c:\Users\MoaviaHassan\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [25]:
from xgboost import XGBRegressor
import numpy as np
from sklearn.metrics import mean_squared_error

# Create an XGBoost Regressor instance with the correct objective and parameters
xgb = XGBRegressor(objective='reg:squarederror', 
                   max_depth=3, learning_rate=0.01, 
                   n_estimators=150, 
                   colsample_bytree=0.5,early_stopping_rounds=10)

xgb.fit(early_stopping_rounds=0)

# Manual implementation of early stopping
best_score = float('inf')
best_iteration = 0
no_improve_rounds = 0
early_stopping_rounds = 50

for i in range(1, xgb.n_estimators + 1):
    # Train the model for `i` estimators
    xgb.set_params(n_estimators=i)
    xgb.fit(X_train, y_train, verbose=False)

    # Evaluate the model on the validation set
    y_pred = xgb.predict(X_test)
    current_score = mean_squared_error(y_test, y_pred, squared=False)  # RMSE

    # Check if the current score is better
    if current_score < best_score:
        best_score = current_score
        best_iteration = i
        no_improve_rounds = 0
    else:
        no_improve_rounds += 1

    # Check for early stopping
    if no_improve_rounds >= early_stopping_rounds:
        print(f"Early stopping triggered at iteration {i}")
        break

# Set the final number of estimators to the best iteration
xgb.set_params(n_estimators=best_iteration)

# Fit the final model with the best number of estimators
xgb.fit(X_train, y_train, verbose=False)

# Make predictions and evaluate the final model
y_pred = xgb.predict(X_test)
final_mse = mean_squared_error(y_test, y_pred)
final_rmse = np.sqrt(final_mse)

print("Best Iteration:", best_iteration)
print("Final Mean Squared Error (MSE):", final_mse)
print("Final Root Mean Squared Error (RMSE):", final_rmse)

# Ensure the model has been trained and trees have been added before plotting feature importance
if xgb.get_booster().best_iteration is not None:
    # Plot feature importance
    from xgboost import plot_importance
    import matplotlib.pyplot as plt

    plt.figure(figsize=(10, 8))
    plot_importance(xgb, height=0.8, importance_type='weight', color='green')
    plt.title("Feature Importance (For PETROLEUM ENERGY PRODUCTS CONSUMPTION BY FUEL-Motor Spirit)")
    plt.show()
else:
    print("Model training did not complete with any trees. Unable to plot feature importance.")


TypeError: XGBModel.fit() got an unexpected keyword argument 'early_stopping_rounds'

In [ ]:
# Predictions
predicted_results = xgb.predict(X_test)
# Calculate error metrics
mse = metrics.mean_squared_error(y_test, predicted_results)
mae = metrics.mean_absolute_error(y_test, predicted_results)
rmse = np.sqrt(metrics.mean_squared_error(y_test, predicted_results))
# Calculate mean absolute percentage error (MAPE)
def mean_absolute_percentage_error(y_true, y_pred):
 y_true, y_pred = np.array(y_true), np.array(y_pred)
 return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
mape = mean_absolute_percentage_error(y_test, predicted_results)
# Calculate mean percentage error (MPE)
mpe = ((y_test - predicted_results) / y_test).mean() * 100
# Calculate R-squared (R2) score
r2 = metrics.r2_score(y_test, predicted_results)
# Plotting graph for test and Predicted
plt.figure(figsize=(15, 5))
plt.plot(list(y_test), label='Actual')
plt.plot(list(predicted_results), label='Predicted')
plt.title("Actual vs Predicted [All Sectors]")
plt.ylabel("Demand")
plt.legend()
# Add error metrics as text annotations on the plot
plt.annotate(f'MAPE: {mape:.2f}%', xy=(0.05, 0.70), xycoords='axes fraction')
plt.annotate(f'MPE: {mpe:.2f}%', xy=(0.05, 0.65), xycoords='axes fraction')
plt.annotate(f'R2: {r2:.2f}', xy=(0.05, 0.60), xycoords='axes fraction')
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
# Create features from datetime index
def create_features(df):
    df['date'] = df.index
    df['hour'] = df['date'].dt.hour
    df['dayofweek'] = df['date'].dt.dayofweek
    df['quarter'] = df['date'].dt.quarter
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['dayofyear'] = df['date'].dt.dayofyear
    df['dayofmonth'] = df['date'].dt.day
    df['weekofyear'] = df['date'].dt.weekofyear

    X = df[['hour','dayofweek','quarter','month','year',
           'dayofyear','dayofmonth','weekofyear']]
    return X
# Define your training data
X_train = create_features(df)
y_train = df['domestic']
# Define your XGBoost model
xgb_model = XGBRegressor(objective='reg:squarederror', max_depth=4, learning_rate=0.03, n_estimators=250, colsample_bytree=0.7)
# Train your XGBoost model
xgb_model.fit(X_train, y_train)
# Create features for future dates (adjust this as needed)
future_dates = pd.date_range(start=df.index[-1], periods=36, freq='M') # Assuming you want to forecast 365 days into the future
df_future_dates = pd.DataFrame(index=future_dates)
X_test_future = create_features(df_future_dates)
# Predict future results
predicted_results_future = xgb_model.predict(X_test_future)
# Plot the predicted results
plt.figure(figsize=(13, 8))
plt.plot(predicted_results_future)
plt.title("Predicted Future Sale")
plt.ylabel("domestic ")
plt.legend(('Predicted',))
plt.show()
# Add the predicted results to your future dates DataFrame copy
df_future_dates_copy = df_future_dates.copy()
df_future_dates_copy['Prediction'] = predicted_results_future
# Concatenate the predicted results with your original data
Irr_all_future = pd.concat([df, df_future_dates_copy], sort=False)
# Plot the combined data and predictions
Overview_Complete_Data_And_Prediction_future = Irr_all_future[['domestic', 'Prediction']].plot(figsize=(15, 5))
plt.title("Combined Data and Predictions (Including Future)")
plt.ylabel("Demand")
plt.show()

In [ ]:
predicted_results_future

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assuming you have a Pandas Series 'f1' from your previous calculations
f1 = predicted_results_future


# Reshape 'f1_array' to a 2D array with one column
f1 = f1.reshape(-1, 1)
f1

In [ ]:
import pandas as pd
import numpy as np

# Assuming you have the 'forecasted_values_first' array
forecasted_values_first = np.array([[20022.764],
       [19343.422],
       [19761.19 ],
       [19615.47 ],
       [20679.953],
       [22981.383],
       [29576.69 ],
       [37005.305],
       [30973.584],
       [26211.174],
       [21430.709],
       [20825.102],
       [19873.13 ],
       [19622.541],
       [19761.19 ],
       [19597.936],
       [20696.848],
       [22964.389],
       [30202.451],
       [37700.363],
       [30972.018],
       [26248.355],
       [22060.287],
       [20825.102],
       [19875.209],
       [19644.533],
       [19624.412],
       [19424.512],
       [20723.473],
       [22936.963],
       [30402.096],
       [36687.18 ],
       [30581.762],
       [25799.37 ],
       [21960.703],
       [20340.5  ]], dtype=float)
# Create a DataFrame with the 'Projected Daily Demand' column
data = pd.DataFrame({'Projected Sale': forecasted_values_first.ravel()})

# Print the first few rows of the new dataset
print(data.head())

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Plot only forecasted values
plt.figure(figsize=(15, 5))
plt.plot(f1, label='Projected Values', linestyle='-', color='purple')

# Add gridlines
plt.grid(True)

# Add legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
f2 = df1[150:186]

# Add 'Forecasted_Values' column to 'f2' and populate it with forecasted values
f2['Projected Sale'] = data['Projected Sale'].values
f2

In [ ]:
import matplotlib.pyplot as plt

# Assuming 'f2' is a DataFrame containing 'Forecasted_Values' and 'Demand' columns

# Calculate MAPE and MPE
mape = ((f2['domestic'] - f2['Projected Sale']) / f2['domestic']).abs().mean() * 100
mpe = ((f2['domestic'] - f2['Projected Sale']) / f2['domestic']).mean() * 100

# Plot the 'Forecasted_Values' and 'Demand' series
plt.figure(figsize=(9, 3))
plt.plot(f2.index, f2['Projected Sale'], label='Projected Sale', linestyle='-')
plt.plot(f2.index, f2['domestic'], label='domestic', linestyle='-')

# Add labels and legend
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()

# Add MAPE and MPE as text annotations with adjusted positions
mape_x = f2.index[10]  # Adjust the x-coordinate for MAPE
mape_y = f2['domestic'].max() - 1500  # Adjust the y-coordinate for MAPE
plt.text(mape_x, mape_y, f'MAPE: {mape:.2f}%', fontsize=16, color='red')

mpe_x = f2.index[20]  # Adjust the x-coordinate for MPE
mpe_y = f2['domestic'].max() - 1500  # Adjust the y-coordinate for MPE
plt.text(mpe_x, mpe_y, f'MPE: {mpe:.2f}%', fontsize=16, color='blue')

# Show the plot
plt.grid(True)
plt.tight_layout()
plt.show()